In [28]:
import modern_robotics as mr
import sympy as sp
import numpy as np
from sympy.physics.mechanics import dynamicsymbols, mechanics_printing
mechanics_printing()
from Utilities.symbolicFunctions import*
from Utilities.kukaKinematics import*
from Utilities.RobotClass import*

# TASK 2

### 1. Using the original Denavit-Hartenberg convention, determine the Denavit-Hartenberg parameters for the Agilus robot.

In [29]:
th1, th2, th3, th4, th5, th6 = dynamicsymbols('theta_1, theta_2, theta_3, theta_4, theta_5, theta_6')
al1,al2,al3,al4,al5,al6,al7 = sp.symbols('alpha_1,alpha_2,alpha_3,alpha_4,alpha_5,alpha_6,alpha_7,')

sicConfig = sp.Matrix([[ 25, sp.pi/2 , -400,   th1],
                    [455, 0       ,    0 ,th2],
                    [ 35, sp.pi/2 ,    0,th3 - sp.pi/2],
                    [  0, -sp.pi/2, -420, th4],
                    [  0, sp.pi/2 ,    0,th5],
                    [  0, 0       ,    0,th6]])
sicConfig

⎡      π               ⎤
⎢25    ─   -400    θ₁  ⎥
⎢      2               ⎥
⎢                      ⎥
⎢455   0    0      θ₂  ⎥
⎢                      ⎥
⎢      π              π⎥
⎢35    ─    0    θ₃ - ─⎥
⎢      2              2⎥
⎢                      ⎥
⎢     -π               ⎥
⎢ 0   ───  -420    θ₄  ⎥
⎢      2               ⎥
⎢                      ⎥
⎢      π               ⎥
⎢ 0    ─    0      θ₅  ⎥
⎢      2               ⎥
⎢                      ⎥
⎣ 0    0    0      θ₆  ⎦

#### Here, the first matrix represent the DH parameters in the same formate as in Modern Robotics. The second matrix represent the same table is illustrated as illustrated in Sicilliano. Theta are the free variables of each joint, where we add an additional pi/2 to the third joint to get a get the desired pose illustrated in the report. 

### 2. Determine the end-effector zero position configuration M ∈ SE(3) for the Agilus robot

In [30]:
config1 = sicConfig.subs({th1:0,th2:0,th3:0,th4:0,th5:0,th6:0})
T0i = T_from_sic(config1)               #List of T0i's, T01, T02, T03,...,T06
Tsi = [rotX(sp.pi)*T for T in T0i ]   #List of Tsi's, T1, T2,...,T6

In [31]:
M_DH = Tsi[5] * Tne

M_DH, Me

⎛⎡1  0  0  980⎤  ⎡1  0  0  980⎤⎞
⎜⎢            ⎥  ⎢            ⎥⎟
⎜⎢0  1  0   0 ⎥  ⎢0  1  0   0 ⎥⎟
⎜⎢            ⎥, ⎢            ⎥⎟
⎜⎢0  0  1  435⎥  ⎢0  0  1  435⎥⎟
⎜⎢            ⎥  ⎢            ⎥⎟
⎝⎣0  0  0   1 ⎦  ⎣0  0  0   1 ⎦⎠

####  Control: The M derived by visual inspection in Utilities/KukaKinematics and the Slist derived from the DH-formulation are in agreement. 
#### To compansate for the additional rotation of pi/2 in the table, we rotate the third joint with -pi/2 to get the zero-state configuration. As M represent the transformation matrix from the {s} frame to the end effector, and since the table from task one only represent the parameters of each joint, we add an additional T from the n'th link to the end-effector. 

### 3. Determine the space frame screw axes Si for the Agilus robot.

In [32]:
Ai = sp.Matrix([[0,-1,0,0],[1,0,0,0],[0,0,0,0],[0,0,0,0]]) # This is a given matrix due to revolute joints

# S_sp = sp.zeros(6)
# for i in range(6):
#     dot_sum = sp.eye(4)
#     for n in range(i,-1,-1):
#         dot_sum = Tlist_DH[i] * dot_sum
#     S_skew = dot_sum * Ai * sp.Inverse(dot_sum)
#     S_sp[0,i] = S_skew[2,1] 
#     S_sp[1,i] = S_skew[0,2] 
#     S_sp[2,i] = S_skew[1,0] 
#     S_sp[3,i] = S_skew[0,3] 
#     S_sp[4,i] = S_skew[1,3]
#     S_sp[5,i] = S_skew[2,3] 

# S_sp, sp.Matrix(Slist)

####  Control: The Slist derived by visual inspection in Utilities/KukaKinematics and the Slist derived from the DH-formulation are in agreement. 

### 4. Determine the body-frame screw axes Bi for the Agilus robot

In [33]:
Mb = M6*Tnb                     #Mb = Tsb for robot in zero-config
M_inv = mr.TransInv(Mb)         #Finding inverse of M
Ad_M_inv = mr.Adjoint(M_inv)    #Computing [Ad_M^-1]

#Using B_i = [Ad_M^-1]S_i
Blist2 = sp.zeros(6,6)
for i in range(6):
     Blist2[:, i] = Ad_M_inv @ Slist[:, i]

Blist, Blist2


⎛⎡ 0     0     0    -1  0  -1⎤  ⎡  0       0       0     -1.0   0   -1.0⎤⎞
⎜⎢                           ⎥  ⎢                                       ⎥⎟
⎜⎢ 0     1     1    0   1  0 ⎥  ⎢  0      1.0     1.0     0    1.0   0  ⎥⎟
⎜⎢                           ⎥  ⎢                                       ⎥⎟
⎜⎢ -1    0     0    0   0  0 ⎥  ⎢ -1.0     0       0      0     0    0  ⎥⎟
⎜⎢                           ⎥, ⎢                                       ⎥⎟
⎜⎢ 0     35    35   0   0  0 ⎥  ⎢  0      35.0    35.0    0     0    0  ⎥⎟
⎜⎢                           ⎥  ⎢                                       ⎥⎟
⎜⎢-900   0     0    0   0  0 ⎥  ⎢-900.0    0       0      0     0    0  ⎥⎟
⎜⎢                           ⎥  ⎢                                       ⎥⎟
⎝⎣ 0    -875  -420  0   0  0 ⎦  ⎣  0     -875.0  -420.0   0     0    0  ⎦⎠

####  Control: The Blist derived by visual inspection in Utilities/KukaKinematics and calculated Blist are in agreement. 

### 5. Visualization 
#### Visualization using custom class in Utilities module. The big coordinate system being the {s}-frame and joint-frames oriented with z-axis along joint screw-axis. Joint 6 is coloured red
#### Axis colours:
####    -Blue   -Z axis and joint screw-axis
####    -Red    -X axis
####    -Green  -Y axis

#### Visualization of DH frames:

In [34]:
#Finding M01, M02..., from List of Tsi developed in task 2.2
Mlist_DH = np.array(Tsi,dtype=float)

#Visualize frames
KukaDH = Robot(Mlist_DH, ['z','-z','x','y','-z','z'], Tne)
KukaDH.draw_robot()

WebVisualizer(window_uid='window_3')

#### 

#### DH-frames shown as joint-frames. Note that: __Birk insert__ and that all three wrist joints are co-located with {b}. {e} is translated 80mm along x-axis of {b} to show orientation clearly.

In [35]:
KukaDH.transform(Slist,[0,0,np.pi/2,0,0,0])
T = KukaDH.current_config
KukaDH.draw_robot()

WebVisualizer(window_uid='window_4')

#### Robot shown in the config that was used for the DH-analysis

#### Visualization of Kuka robot in zero-config. Showing Me, Si and Bi:
#### Me is the chains last frame, screw axes as the joint frames' z-axis (both Bi and Si).
#### {4} is drawn outside of the wrist to show functionality better.
#### {b} is located inside the wrist(red/grey) and oriented as {e}

In [36]:
Kuka = Robot(Mlist, ['z', '-z', 'x', 'x', '-z','x'], Tne)

In [37]:
Kuka.joints[5].set_colour([1,0,0])
Kuka.joints[4].set_colour([0.2,0.2,0.2])
Kuka.draw_robot()

WebVisualizer(window_uid='window_5')

In [38]:
#Oppgave 2.6

joint_config1 = [0,0,0,0,0,0]
joint_config2 = [0,0,-np.pi/2,np.pi/2,0,0]

M_poe = np.array([[0,0,-1,900],
                [0,1,0,0],
                [1,0,0,435],
                [0,0,0,1]])

T_poe = mr.FKinSpace(M_poe, Slist, joint_config2)
T_poe = T_poe * Tne

print(T_poe)

Matrix([[1.11022302462516e-16, 1.00000000000000, -1.11022302462516e-16, 445.000000000000], [0, 1.11022302462516e-16, 1.00000000000000, 0], [1.00000000000000, -1.11022302462516e-16, 1.23259516440783e-32, 900.000000000000], [0, 0, 0, 1.00000000000000]])


In [39]:
config2 = sicConfig.subs({th1:0,th2:0,th3:-np.pi/2,th4:np.pi/2,th5:0,th6:0}) #DH-parameters with given thetas
T0i = T_from_sic(config2)               #List of T0i's, T01, T02, T03,...,T06
Tsi = [rotX(sp.pi)*T for T in T0i ]     #List of Tsi's, T1, T2,...,T6
Tse = Tsi[5] * Tne
print(Tse)


Matrix([[6.12323399573677e-17, 1.00000000000000, -6.12323399573677e-17, 445.000000000000], [0, 6.12323399573677e-17, 1.00000000000000, 0], [1.00000000000000, -6.12323399573677e-17, 3.74939945665464e-33, 900.000000000000], [0, 0, 0, 1]])


In [46]:
def PoeDHCompare(thetaList):
    config2 = sicConfig.subs({th1:thetaList[0],th2:thetaList[1],th3:thetaList[2],th4:thetaList[3],th5:thetaList[4],th6:thetaList[5]}) #DH-parameters with given thetas
    T0i = T_from_sic(config2)               #List of T0i's, T01, T02, T03,...,T06
    Tsi = [rotX(sp.pi)*T for T in T0i ]     #List of Tsi's, T1, T2,...,T6
    Tse = Tsi[5] * Tne

    T_poe = mr.FKinSpace(M_poe, Slist, thetaList) #PoE 
    T_poe = T_poe * Tne #From joint 5 to end-effector

    Tse = np.array(Tse).astype(np.float64) #Convert from Matrix to Numpy Array
    T_poe = np.array(T_poe).astype(np.float64)
    res = np.allclose(Tse, T_poe, 0.0001) #Compares the arrays with a given resoulution
    
    print("T_DH: \n", Tse)
    print("T_poe: \n", T_poe)
    return res

PoeDHCompare([0,0,0,0,0,0])


T_DH: 
 [[  1.   0.   0. 980.]
 [  0.   1.   0.   0.]
 [  0.   0.   1. 435.]
 [  0.   0.   0.   1.]]
T_poe: 
 [[  1.   0.   0. 980.]
 [  0.   1.   0.   0.]
 [  0.   0.   1. 435.]
 [  0.   0.   0.   1.]]


True

In [47]:
PoeDHCompare([0,0,-np.pi/2,np.pi/2,0,0])

T_DH: 
 [[ 6.12323400e-17  1.00000000e+00 -6.12323400e-17  4.45000000e+02]
 [ 0.00000000e+00  6.12323400e-17  1.00000000e+00  0.00000000e+00]
 [ 1.00000000e+00 -6.12323400e-17  3.74939946e-33  9.00000000e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
T_poe: 
 [[ 1.11022302e-16  1.00000000e+00 -1.11022302e-16  4.45000000e+02]
 [ 0.00000000e+00  1.11022302e-16  1.00000000e+00  0.00000000e+00]
 [ 1.00000000e+00 -1.11022302e-16  1.23259516e-32  9.00000000e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


True

In [48]:
PoeDHCompare([1,1,1,1,1,0])

T_DH: 
 [[-9.40674425e-01  4.12378082e-02  3.36795292e-01 -6.15939938e+00]
 [ 1.54500205e-01  9.35775919e-01  3.16943396e-01 -9.52483768e+01]
 [-3.02094873e-01  3.50175488e-01 -8.86631725e-01 -4.03506946e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
T_poe: 
 [[-9.40674425e-01  4.12378082e-02  3.36795292e-01 -6.15939938e+00]
 [ 1.54500205e-01  9.35775919e-01  3.16943396e-01 -9.52483768e+01]
 [-3.02094873e-01  3.50175488e-01 -8.86631725e-01 -4.03506946e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


True

In [49]:
PoeDHCompare([np.pi/4,0,0,0,0,0])

T_DH: 
 [[   0.70710678    0.70710678    0.          692.96464556]
 [  -0.70710678    0.70710678    0.         -692.96464556]
 [   0.            0.            1.          435.        ]
 [   0.            0.            0.            1.        ]]
T_poe: 
 [[   0.70710678    0.70710678    0.          692.96464556]
 [  -0.70710678    0.70710678    0.         -692.96464556]
 [   0.            0.            1.          435.        ]
 [   0.            0.            0.            1.        ]]


True

In [50]:
PoeDHCompare([0,0,0,0,0,np.pi/2])

T_DH: 
 [[ 1.000000e+00  0.000000e+00  0.000000e+00  9.800000e+02]
 [ 0.000000e+00  6.123234e-17  1.000000e+00  0.000000e+00]
 [ 0.000000e+00 -1.000000e+00  6.123234e-17  4.350000e+02]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]
T_poe: 
 [[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  9.80000000e+02]
 [ 0.00000000e+00  1.11022302e-16  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -1.00000000e+00  1.11022302e-16  4.35000000e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


True